In [1]:
from Arduino_SPI_bridge import SPI_Bridge
from struct import pack
from time import sleep

def readWriteReg(isRead=True, adr=0, payload=[0], cs=8):
    ''' 
    do a SPI transaction with Arduino_SPI_bridge
    cs: chip select pin DAC = 8 HMC = 9
    '''
    adr &= 0xFFFF
    pl = [(isRead << 7) | (adr >> 8), adr & 0xFF]
    pl += payload
    res = b.sendReceive(cs, bytes(pl))
    if res[0]:
        res = res[1][2:-1]  # Extract actual SPI payload
        return res[2:]      # Extract chip reply
    else:
        return bytes()

def rr(adr=0, l=1):
    ''' read a register '''
    dat = readWriteReg(True, adr, [0] * l)
    if len(dat) == 1:
        dat = dat[0]
    return dat
    
def wr(adr, dat):
    ''' write a register '''
    if type(dat) is int:
        dat = [dat]
    return readWriteReg(False, adr, dat)

def hd(dat):
    ''' print a hex-dump '''
    for i, d in enumerate(dat):
        if i % 8 == 0 and len(dat) > 8:
            print('\n{:04x}: '.format(i), end='')
        print('{:02x} '.format(d), end='')
    print()

In [3]:
b = SPI_Bridge('/dev/ttyUSB0')
b.configSPI(baudrate=1000000, order=b.MSBFIRST, mode=b.SPI_MODE0)

send:    0x09 0x00 0x00 0x0f 0x42 0x40 0x01 0x00 0xfa 
receive: 0x03 0x01 0xfd 


True

# Power up sequence

In [376]:
# Power up sequence, Table 51
wr(0x000, 0x81)  # Soft reset

wr(0x000, 0x3C)  # 4 - wire SPI mode
wr(0x091, 0x00)  # Power up clock RX
# wr(0x206, 0x01)  # Enable PHYs
wr(0x705, 0x01)  # Enable boot loader
wr(0x090, 0x00)  # Power on DACs and bias supply

# Disable DAC PLL and config for external clock, Table 52
wr(0x095, 0x01)
wr(0x790, 0xFF)
wr(0x791, 0xFF)

wr(0x008, (1 << 7) | (1 << 6)) # Select both DACs

# Magic numbers from Table 54 (calibration)
# wr(0x050, 0x2A)
# wr(0x061, 0x68)
# wr(0x051, 0x82)
# wr(0x051, 0x83)
# print('CAL_STAT:', rr(0x052))
# wr(0x081, 0x03)  # Power down calibration clocks

# JESD config, Table 55
wr(0x100, 0x00)  # Power up digital datapath clocks
wr(0x201, 0xFF)  # Power down unused PHYs.

# Setup DDSes
wr(0x1E6, (1 << 1))             # Enable DDSM_EN_CAL_DC_INPUT (see Fig. 80) (tone on / off)
wr(0x112, (1 << 3) | (1 << 2))  # Enable NCO + Modulus
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

bytearray(b'\x00')

In [405]:
def setTone(dac_select=1, f_out=None, ampl=None, del_a=None, mod_b=None, phase=None, f_ref=5125e6):
    '''
    dac_select: 1 = first DAC, 2 = second DAC, 3 = both DACs
    '''
    # Select a DAC main datapath
    dac_select &= 0x03
    
    if ampl is not None:
        # Set DC amplitude level (2 bytes), 0x50FF is full-scale tone
        # Updates immediately without the need for DDSM_FTW_LOAD_REQ
        # TODO: not clear if synchronized to 16 bit write
        ampl_b = int(min(ampl, 1.0) * 0x50FF).to_bytes(2, 'little')
        wr(0x008, dac_select)  # need to use __CHANNEL_PAGE_ !!!!
        wr(0x148, ampl_b)
        print('DC_CAL_TONE: ', end='')
        hd(rr(0x148, 2))
    
    # All other regs are on MAINDAC_PAGE
    wr(0x008, (dac_select << 6))

    if f_out is not None:
        # ftw updates on posedge DDSM_FTW_LOAD_REQ
        ftw_b = int(f_out / f_ref * 2**48).to_bytes(6, 'little')  # [Hz]
        wr(0x114, ftw_b)  # Write 6 bytes FTW into main NCO
        print('DDSM_FTW: ', end='')
        hd(rr(0x114, 6))

    if phase is not None:
        # DDSM_NCO_PHASE_OFFSET updates immediately without DDSM_FTW_LOAD_REQ
        # However it updates after each 8th risign edge on the SPI clock (see scope shot)
        # it does not synchronize to the 16 bit register width :(
        # --> phase jump of ~1.4 degree on register rollover is unavoidable :(
        phase_b = int(phase / 180 * 2**15).to_bytes(2, 'little', signed=True)  # [deg]
        wr(0x11C, phase_b)  # Write 2 bytes
        print('DDSM_NCO_PHASE_OFFSET: ', end='')
        hd(rr(0x11C, 2))

    if del_a is not None:
        # Modulus and Delta are updated after posedge DDSM_FTW_LOAD_REQ
        # confirmed by scope measurement
        wr(0x12A, del_a.to_bytes(6, 'little'))  # Write 6 bytes Delta [A]
        print('DDSM_ACC_DELTA: ', end='')
        hd(rr(0x12A, 6))
        
        wr(0x124, mod_b.to_bytes(6, 'little'))  # Write 6 bytes Modulus [B]
        print('DDSM_ACC_MODULUS: ', end='')
        hd(rr(0x124, 6))

    if f_out is not None or del_a is not None:
        # Positive edge on DDSM_FTW_LOAD_REQ applies the FTW and causes a phase glitch!
        # Random phase jump -pi .. pi on selected DAC
        wr(0x113, 0x01)  # Update settings
        print('DDSM_FTW_LOAD_ACK:', (rr(0x113) >> 1) & 0x01)
        wr(0x113, 0x00)

In [378]:
setTone(1, f_out=499.6e6, ampl=1)
setTone(2, f_out=499.6e6, ampl=1, del_a=99, mod_b=100)
# setTone(2, del_a=100, mod_b=99)

DC_CAL_TONE: ff 50 
DDSM_FTW: 6a a6 1e a4 f4 18 
DDSM_FTW_LOAD_ACK: 1
DC_CAL_TONE: ff 50 
DDSM_FTW: 6a a6 1e a4 f4 18 
DDSM_ACC_DELTA: 63 00 00 00 00 00 
DDSM_ACC_MODULUS: 64 00 00 00 00 00 
DDSM_FTW_LOAD_ACK: 1


In [403]:
setTone(2, phase=-13)

DDSM_NCO_PHASE_OFFSET: c2 f6 


# Things tried
  * setting DDSM_FTW_UPDATE (0x113) to 0x60 enables auto updating. Writing to DDSM_FTW5 (0x119) applies the FTW and unfortunately also causes a phase glitch :(
  * setting DDSM_FTW_LOAD_SYSREF does not trigger the FTW update, because SYSREF comes from HMC chip which is not active

# More things to try
  * is there a setting to avoid phase glitch on posedge DDSM_FTW_LOAD_REQ
  * does the channel NCO also have this phase glitch
  * use the fast hopping NCO?

In [408]:
wr(0x113, 0x60)

bytearray(b'\x00')

In [424]:
wr(0x008, (2 << 6))
hex(rr(0x113))

'0x60'

In [442]:
hd(rr(0x114, 6))

6a a6 1e a4 f4 17 


In [438]:
wr(0x119, 0x17)

bytearray(b'\x00')

In [443]:
wr(0x113, (1 << 2))

bytearray(b'\x00')